In [2]:
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import StronglyEntanglingLayers
import torch
from torch.autograd import Variable

# Cargar datos
data_train = pd.read_csv('challenge_train.csv')
data_test = pd.read_csv('challenge_test.csv')

# Preparar características y etiquetas
features_train = data_train[['F1', 'F2', 'F3', 'F4']].values
targets_train = data_train['Target'].values
features_test = data_test[['F1', 'F2', 'F3', 'F4']].values
targets_test = data_test['Target'].values

# Normalizar características
min_values = np.min(features_train, axis=0)
max_values = np.max(features_train, axis=0)
features_train_normalized = (features_train - min_values) / (max_values - min_values)
features_test_normalized = (features_test - min_values) / (max_values - min_values)

# Convertir los datos normalizados a tensores de PyTorch
features_train_torch = torch.tensor(features_train_normalized).float()
targets_train_torch = torch.tensor(targets_train).float()
features_test_torch = torch.tensor(features_test_normalized).float()
targets_test_torch = torch.tensor(targets_test).float()

# Configuración del dispositivo cuántico
num_qubits = 4
dev = qml.device('default.qubit', wires=num_qubits)

In [6]:
@qml.qnode(dev, interface='torch')
def quantum_circuit(features, weights):
    qml.templates.AmplitudeEmbedding(features, wires=range(num_qubits), pad_with=0., normalize=True)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# Ajuste de la inicialización de los pesos
num_layers = 2  # Ajustado de 4 a 6 capas
init_weights = np.random.uniform(low=-1, high=1, size=(num_layers, num_qubits, 3))  # Ajuste en la distribución
weights = Variable(torch.tensor(init_weights), requires_grad=True)

# Función de coste ajustada
def cost(weights):
    loss = 0
    reg_lambda = 2.5  # Ajustado a un valor más bajo
    for i in range(len(features_train_torch)):
        f = features_train_torch[i]
        label = targets_train_torch[i]
        prediction = quantum_circuit(f, weights)
        loss += (prediction - label) ** 2
    l2_reg = reg_lambda * torch.sum(weights ** 2)  # Uso de sum para calcular L2
    return (loss / len(features_train_torch)) + l2_reg

# Optimizador ajustado
optimizer = torch.optim.Adam([weights], lr=0.05)  # Ajuste del learning rate

# Bucle de entrenamiento ajustado
steps = 100  # Aumento del número de pasos
for step in range(steps):
    optimizer.zero_grad()
    loss = cost(weights)
    loss.backward()
    optimizer.step()
    if step % 10 == 0:
        print(f"Step {step} - Loss: {loss.item()}")

# Función de predicción ajustada
def predict(features, weights):
    predictions = [quantum_circuit(f, weights).item() for f in features]
    predicted_labels = torch.tensor(predictions).float()
    # Ajuste del umbral de clasificación
    predicted_labels = torch.where(predicted_labels > 0.0, 1, 0).int()
    return predicted_labels

# Evaluar el accuracy
predicted_labels = predict(features_test_torch, weights)
accuracy = torch.mean((predicted_labels == targets_test_torch).float())
print(f'Accuracy: {accuracy.item()*100}%')

Step 0 - Loss: 25.146556043933625
Step 10 - Loss: 4.099189718582357
Step 20 - Loss: 0.904635497839444
Step 30 - Loss: 1.0067087213294585
Step 40 - Loss: 0.6319579198779275
Step 50 - Loss: 0.5128586602563928
Step 60 - Loss: 0.49227426292604765
Step 70 - Loss: 0.4767682224796805
Step 80 - Loss: 0.4726910943149468
Step 90 - Loss: 0.4710385361375907
Accuracy: 69.9999988079071%
